# Generate database diagram with graphviz
Database diagrams ease understanding relations in database and thus help data users to work more efficient but only if they are well organized which is difficult to achieve automatically without someone manually editing the layout.   

It is usefull to have the ability to generate database schema ad-hoc without need to use some modelling tools.

This simple demo shows how to do it just using python and graphviz with: 
- custom node html based style for tables 
- on hover descriptions 
- on click linking

Check the demo result here: [my_database.gv.svg](my_database.gv.svg)

You can also check how it looks with more tables here: [https://storage.googleapis.com/data_catalog_diagram_extera_presale/extera_nexus.gv.svg](https://storage.googleapis.com/data_catalog_diagram_extera_presale/extera_nexus.gv.svg)  

as a part of [MindForce Extera](https://www.mindforce.ai/extera) Data Catalog: [https://datastudio.google.com/u/0/reporting/1D5pVDr_OCVxwL2YpC3WPwBMoepvRa2HI/page/6nVl](https://datastudio.google.com/u/0/reporting/1D5pVDr_OCVxwL2YpC3WPwBMoepvRa2HI/page/6nVl) 

## Set Up

1) download and install graphviz: [https://graphviz.org/download/source/](https://graphviz.org/download/source/)
2) install python packages

In [ ]:
!pip install pandas 
!pip install graphviz
!pip install openpyxl #to read xlsx

In [ ]:
import pandas as pd
from graphviz import Digraph
import re

## Get metadata
There are 3 metadata entities used for schema generation
1. Tables - description
2. Table columns + description, data type, primary key, column order
3. Table relations - related tables, key columns

In this example metadata are retrieved from sample excel file in this folder.

In [ ]:
tables = pd.read_excel('database_metadata.xlsx',sheet_name='tables')
columns = pd.read_excel('database_metadata.xlsx',sheet_name='columns')
relations = pd.read_excel('database_metadata.xlsx',sheet_name='relations')

## Define graph node look and prepare data model and settings for graphviz

In [ ]:
#html tamplate for graph nodes
def gen_html_table_node(project_id,dataset_id,table_id):
    table_cols = columns[(columns['project_id']==project_id)&(columns['dataset_id']==dataset_id)&(columns['table_id']==table_id)]
    if re.search('^c_',table_cols['table_id'].iloc[0]) is not None:
        #codetable background color
        table_color = '#FFFF99'
    else:
        #table background color
        table_color = '#99FFCC'
    cols_string = ''
    for _, row in table_cols.iterrows():
        port = row['table_id']+row['column_name']
        column_name = row['column_name']
        column_data_type = row['data_type']
        table_description = row['table_description_cz']
        # link to datacatalog table detail, in this demo just mindforce web 
        table_href = 'https://en.mindforce.ai/'
        column_string = '''<TR>
        <TD PORT="{port}" ALIGN="LEFT">{column_name}</TD>
        <TD ALIGN="CENTER"><i>{column_data_type}</i></TD>
        </TR>'''.format(port=port,column_name=column_name, column_data_type=column_data_type)
        cols_string = cols_string+column_string
    table_begin_string = '''<
    <TABLE HREF="{}" TITLE = "{}" BGCOLOR="{}" BORDER="0" CELLBORDER="1" CELLSPACING="1" CELLPADDING="1">
    <TR><TD PORT="{}" COLSPAN= "2"><b>{}</b></TD></TR>'''.format(table_href,table_description,table_color,row['table_id'],row['table_id'])
    table_end_string = '''</TABLE>>'''
    table_node = table_begin_string+cols_string+table_end_string
    return table_node 

# prepare data for graph rendering       
def generate_ER_dataset_diagram(project_id, dataset_id):
    # graph initialization a graph settings
    dataset_graph = Digraph(dataset_id, format='svg', node_attr={'shape': 'plaintext'}, graph_attr={'splines':'true','overlap':'false'}, engine = 'fdp')
    dataset_tables = tables[(tables['dataset_id']==dataset_id)&(tables['project_id']==project_id)]
    # adding nodes
    for _, row in dataset_tables.iterrows():
        table_id = row['table_id']
        node_html = gen_html_table_node(project_id,dataset_id,table_id)
        dataset_graph.node(table_id,node_html,style='rounded')
    # adding edges
    dataset_relations = relations[(relations['dataset_id']==dataset_id)&(relations['project_id']==project_id)]
    for _, row in dataset_relations.iterrows():
        edge_name = row['column_name']
        parent_string = '{}'.format(row['table_id'])
        child_string = '{}'.format(row['ref_table_id'])
        dataset_graph.edge(parent_string, child_string,edge_name)
    return dataset_graph

## Render diagram

In [ ]:
PROJECT_ID = 'my_project'
DATASET_ID = 'my_database'
# render graph with graphviz
diag = generate_ER_dataset_diagram(PROJECT_ID,DATASET_ID)
diag.render()